In [ ]:
# importing the libraries
import os
import gzip
import numpy as np
from sklearn import svm
from sklearn.model_selection import train_test_split
from scipy.optimize import minimize
from cvxopt import matrix, solvers
from time import time

In [ ]:
# importing the data
def load_mnist(path, kind='train'):

    """Load MNIST data from `path`"""
    labels_path = os.path.join(path,
                               '%s-labels-idx1-ubyte.gz'
                               % kind)
    images_path = os.path.join(path,
                               '%s-images-idx3-ubyte.gz'
                               % kind)

    with gzip.open(labels_path, 'rb') as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8,
                               offset=8)

    with gzip.open(images_path, 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8,
                               offset=16).reshape(len(labels), 784)

    return images, labels


cwd = os.getcwd()

X_all_labels, y_all_labels = load_mnist(cwd, kind='train')

"""
We are only interested in the items with label 2, 4 and 6.
Only a subset of 1000 samples per class will be used.
"""
indexLabel3 = np.where((y_all_labels==3))
xLabel3 =  X_all_labels[indexLabel3][:1000,:].astype('float64')
yLabel3 = y_all_labels[indexLabel3][:1000].astype('float64')

indexLabel8 = np.where((y_all_labels==8))
xLabel8 =  X_all_labels[indexLabel8][:1000,:].astype('float64')
yLabel8 = y_all_labels[indexLabel8][:1000].astype('float64')

In [ ]:
# preparing the data
x_con = np.concatenate((xLabel3, xLabel8), axis = 0)
y_con = np.concatenate((yLabel3, yLabel8))

#normalization of values in x dataset 
x_con = x_con /255

#changing the labels 
y_con[y_con <= 3] = -1; y_con[y_con >= 8] = 1

In [ ]:
x_tr, x_test, y_tr, y_test = train_test_split(x_con, y_con, train_size = 0.8, test_size = 0.2, random_state = 1705471)

# SVM dual quadratic problem

**Some functions to handle some operations later**

In [ ]:
def R(alpha,c):
  Lp = np.where((alpha <= 1e-06)&(y==1))[0]                 #first condition
  Um = np.where((alpha >= C -(1e-06))&(y==-1))[0]           #second condition
  inside = np.where((alpha > 1e-06)&(alpha < C-(1e-06)))[0] #third condition
  r = np.concatenate((Lp,Um,inside)).astype(int)
  r = np.sort(r)
  return r

def S(alpha,c):
  Lp = np.where((alpha <= 1e-06)&(y==-1))[0]                #first condition
  Um = np.where((alpha >= C -(1e-06))&(y==1))[0]            #second condition
  inside = np.where((alpha > 1e-06)&(alpha < C-(1e-06)))[0] #third condition 
  s = np.concatenate((Lp,Um,inside)).astype(int)
  s = np.sort(r)
  return s

**SVM class**

In [ ]:
class customSVM(object):
    
    # init funtion
    def __init__(self, x, y, C, gamma):
        self.x = x; self.y = y
        self.C = C; self.gamma = gamma
        self.alpha = np.zeros(y.shape[0])
    
    # funtion to get all the parameters
    def get_par(self):
        return self.x, self.y, self.C, self.gamma, self.alpha
    
    # set function
    def set_par(self, x, y, C, gamma, alpha):
        self.x = x; self.y = y
        self.C = C; self.gamma = gamma
        self.alpha = alpha
        
    # linear kernel
    def lin_ker(self):
        return(np.power(self.x.dot(self.x.T) + 1, self.gamma))
    
    # calculating the alpha by minimizing the lagrangian for the soft SVM

    def opt_alpha(self):

      x, y, C, gamma, alpha = self.get_par()
      n_samples, n_features = x.shape
      start_time = time()
      
      # Compute the Gram matrix

      K = np.zeros((n_samples, n_samples))

      '''
      for i in range(n_samples):
        for j in range(n_samples):
          K[i,j] = self.lin_ker(x[i], x[j])
      '''

      # costruction of the function to minimize

      P = matrix(np.outer(y,y) * self.lin_ker())
      q = matrix(np.ones(n_samples) * -1)

      A = matrix(y, (1,n_samples))
      b = matrix(0.0)

      G = matrix(np.vstack((np.diag(np.ones(n_samples) * -1), np.identity(n_samples))))
      h = matrix(np.hstack((np.zeros(n_samples), np.ones(n_samples) * C)))

      # solve QP problem
      solvers.options['show_progress'] = False
      solution = solvers.qp(P, q, G, h, A, b)

      # updating with the final value of alpha
      self.set_par(x, y, C, gamma, np.array(solution['x']))
      #print(self.lin_ker()[0, 0])
      return solution['primal objective'], solution['iterations'], time() - start_time

**K-Fold CV to proper set C and gamma hyper parametres**

In [ ]:
hyperPar = {
    'C': [0.01,0.015,0.02,0.1,0.15,0.2,0.5,1,2,5],
    'gamma': [1,1.75,2]
}

hyperPar1 = {
    'C': [0.1],
    'gamma': [1]
}

opt_C = 0; opt_gamma = 0; opt_alpha = 0; opt_b = 0
opt_acc = 0; opt_in_fun_val = 0; tr_acc = 0
tot_it = 0; tot_time = 0
k = len(hyperPar['C']) * len(hyperPar['gamma'])


alpha_res = []

for c in hyperPar1['C']:
    for Gamma in hyperPar1['gamma']:
        # creation of the SVM object
        accl=[]
        n_split = 5
        size = int(len(x_con) / n_split)        
        for iteration in range(n_split):
            if iteration==0:
                x_test = x_con[:size]
                y_test = y_con[:size]
                x_tr = x_con[size:]
                y_tr = y_con[size:]
            else:
                #qua è necessario farlo sulla stessa riga sennò x_test sfattona
                x_test, x_tr = x_tr[:size], np.concatenate((x_tr[size:],x_test)) 
                y_test, y_tr = y_tr[:size],  np.concatenate((y_tr[size:],y_test)) 
            # changing labels
            
            
            tr_SVM = customSVM(x_tr, y_tr, c, Gamma)
            #a ogni iterazione test si aggiorna del size del bucket, mentre x_train si riduce del bucket andato in test e riceve x_test precedente
        
        # computation of the optimal point and extraction of the result

            fun_val, n_it, ex_time = tr_SVM.opt_alpha()
            x, y, C, gamma, alpha = tr_SVM.get_par()
            tot_it += n_it; tot_time += ex_time
            
            # weights and biases

            w = np.multiply(alpha.reshape(y.shape[0]), y).dot(x)   #calcolo w con alpha ottimale 
            b = np.average(y - w.dot(x.T))                         #calcolo b con w e alpha ottimali
            
            # calculation of the accuracy to choose the best params

            pred = np.sign(w.dot(x_test.T) + b)              # we use sign function to predict on k-foldt test
            acc = np.sum(pred == y_test) / y_test.shape[0]   # computing accuracy related to k- fold test
            pred_tr = np.sign(w.dot(x_tr.T) + b)            
            acc_tr = np.sum(pred_tr == y_tr) / y_tr.shape[0]
            #alpha_res.append(alpha)

            accl.append(acc)

        if np.mean(accl) > opt_acc:
          opt_C = c; opt_gamma = Gamma; opt_acc = np.mean(accl); tr_acc = acc_tr
          print(np.mean(accl))
          print('optimal values:', 'gamma:',opt_gamma,'C:',opt_C)
          #opt_in_fun_val = tr_SVM.f(alpha, np.outer(y,y) * tr_SVM.lin_ker(), np.ones((x.shape[0], 1)) * -1)
          opt_alpha = alpha; opt_b = b

0.9765
optimal values: gamma: 1 C: 0.1


**Here we train the model with optimal values of C,gamma again. Then check all kkt violations, and finally check if the value of optimal alpha we evaluated  is a kkt point**

In [ ]:
kkt_viol = 0; opt_alpha = opt_alpha.reshape((1600))

kkt_SVM = customSVM(x_tr, y_tr, opt_C, opt_gamma) # on optimal values of C and alpha

rho = np.multiply(opt_alpha.reshape(y_tr.shape[0]), y_tr).dot(kkt_SVM.lin_ker())
print(rho)

rho += y_tr - np.average(rho)

rho_y = np.multiply(y_tr, rho)

# calculating kkt violations

g = 5*10**(-2) #gap for points really close to 0 or C value

for i in range(rho_y.shape[0]):
    if rho_y[i] == 1 and (opt_alpha[i] >= opt_C-g or opt_alpha[i] <= 0+g):
        kkt_viol += 1
        
    if rho_y[i] < 1 and opt_alpha[i] != opt_C:
        kkt_viol += 1
        
    if rho_y[i] > 1 and opt_alpha[i] != 0:
        kkt_viol += 1

# calculation of the kkt optimality
# we set a gap a less grater than zero, otherwise we'll select
# all indeces both for S and R

#calculating R set

Lp = np.where((opt_alpha <= g)&(y==1))[0]                      #first condition
Um = np.where((opt_alpha >= C -(g))&(y==-1))[0]                #second condition
#inside = np.where((opt_alpha > g)&(opt_alpha < C-(g)))[0]     #third condition
r = np.concatenate((Lp,Um)).astype(int)
r = np.sort(r)

# calculating S set

Lp = np.where((opt_alpha <= g)&(y==-1))[0]                     #first condition
Um = np.where((opt_alpha >= C -(g))&(y==1))[0]                 #second condition
#inside = np.where((opt_alpha > g)&(opt_alpha < C-(g)))[0]     #third condition 
s = np.concatenate((Lp,Um)).astype(int)
s = np.sort(s)

delta_f = (np.outer(y_tr, y_tr) * kkt_SVM.lin_ker()).dot(opt_alpha) - np.ones(opt_alpha.shape[0])

print(r == s) #check if R and S set are different 

m = np.max(-np.multiply(delta_f[r[0]], y_tr[r[0]]))
M = np.min(-np.multiply(delta_f[s[0]], y_tr[s[0]]))

**In this part we print all relevant information**

In [ ]:
# printing the results
print("Optimal C: ", opt_C)
print("Optimal gamma: ", opt_gamma)
print("Optimization solver: cvxopt qp solver")
print("Mean iteration number: ", tot_it / k)
print("Tot iteration number: ", tot_it )
print("Mean function ev.: ", tot_it / k)
print("Mean time optimization: ", tot_time / k, "s")
print("Train accuracy: ", tr_acc)
print("Test accuracy: ", opt_acc)
print("value of m(a) - M(a) for optimal alpha: ", m - M)

Optimal C:  0.1
Optimal gamma:  1
Optimization solver: cvxopt qp solver
Mean iteration number:  1.8333333333333333
Tot iteration number:  55
Mean function ev.:  1.8333333333333333
Mean time optimization:  0.9648756106694539 s
Train accuracy:  0.98625
Test accuracy:  0.9765
value of m(a) - M(a) for optimal alpha:  -1.555067403399435


# Decomposition method for the dual quadratic problem with q ≥ 4.

#### Here we use **q=10** basing our choise looking to arguments treated during lessons

In [ ]:
tr_SVM = customSVM(x_tr, y_tr, opt_C, opt_gamma)
k = 0; num_features = x_tr.shape[1]; q = 10
alpha = np.zeros(y_tr.shape[0])
alpha_app = np.zeros(y_tr.shape[0])
alpha_grad = -1 * np.ones(y_tr.shape[0])
tot_it = 0; tot_fev = 0; tot_time = 0
Q = np.outer(y_tr, y_tr) * tr_SVM.lin_ker()
val_tol = 0.05

In [ ]:
class customSVMlight(object):
    
    # init funtion
    def __init__(self, x, y, C, gamma, indexes, alpha):
        self.x = x; self.y = y
        self.C = C; self.gamma = gamma
        self.I = indexes
        self.alpha = alpha
    
    # funtion to get all the parameters
    def get_par(self):
        return self.x, self.y, self.C, self.gamma, self.alpha
    
    # set function
    def set_par(self, x, y, C, gamma, alpha):
        self.x = x; self.y = y
        self.C = C; self.gamma = gamma
        self.alpha = alpha
        
    # linear kernel
    def lin_ker(self, x):
        return(np.power(x.dot(x.T) + 1, self.gamma))
    
    # calculating the alpha by minimizing the lagrangian for the soft SVM
    def opt_alpha(self):
        x, y, C, gamma, alpha = self.get_par()
        I = self.I; m = len(I)
        not_I = np.setdiff1d(np.indices(alpha.shape), I)
        y_I = y[I]; Q = np.outer(y, y) * self.lin_ker(x)
        val_b = -np.sum(alpha[not_I] * y[not_I])
        start_time = time()
        
        # costruction of the function to minimize
        P = matrix(Q[I, :][:, I])
        q = matrix(np.full(m, -1) + (Q[I, :][:, not_I]).dot(alpha[not_I]))
        
        # contruction of the matrix for the disequation part          
        G = matrix(np.vstack((np.eye(m) * -1, np.eye(m))))        
        h = matrix(np.hstack((np.zeros(m), np.full(m, C))))
        
        # contruction of the matrix for the equation part
        A = matrix(y_I.reshape(1, -1))
        b = matrix(np.array([val_b]))
        
        # updating with the final value of alpha
        solvers.options['show_progress'] = False
        solution = solvers.qp(P, q, G, h, A, b)
        self.set_par(x, y, C, gamma, np.array(solution['x']).reshape(m))
        return solution['primal objective'], solution['iterations'], time() - start_time

In [ ]:
m =1
M = -1

while m - M > val_tol:
    # selcting the two working set
    R_a = np.where(((alpha < opt_C - val_tol) & (y_tr == 1) | (alpha > val_tol) & (y_tr == -1)))[0]
    S_a = np.where(((alpha < opt_C - val_tol) & (y_tr == -1) | (alpha > val_tol) & (y_tr == 1)))[0]
    
    I_a = np.argsort(- alpha_grad[R_a] * y_tr[R_a])[::-1]
    J_a = np.argsort(- alpha_grad[S_a] * y_tr[S_a])
    W = np.concatenate((I_a[: int(q/2)], J_a[: int(q/2)]))
    
    m = - alpha_grad[R_a[I_a[0]]] * y_tr[R_a[I_a[0]]]
    M = - alpha_grad[S_a[J_a[0]]] * y_tr[S_a[J_a[0]]]
    
    # solving the problem for the chosen W
    W_SVM = customSVMlight(x_tr, y_tr, opt_C, opt_gamma, np.concatenate((R_a[I_a[: int(q/2)]], S_a[J_a[: int(q/2)]])), alpha)
    fun_val, n_it, ex_time = W_SVM.opt_alpha()
    tot_it += n_it; tot_time += ex_time
    
    # updating alpha
    x, y, C, gamma, alpha_W = W_SVM.get_par()
    
    for i in range(int(q/2)):
        alpha[R_a[W[i]]] = alpha_W[i]
        alpha[S_a[W[i + int(q/2)]]] = alpha_W[i + int(q/2)]
        
    # computing the new gradient incrementing k and calculating the new m(a) and M(a)
    alpha_grad += (alpha - alpha_app).dot(Q)
        
    for i in range(int(q/2)):
        alpha_app[R_a[W[i]]] = alpha_W[i]
        alpha_app[S_a[W[i + int(q/2)]]] = alpha_W[i + int(q/2)]
    
    k += 1;
print('Iterations:',k)

Iterations: 309


In [ ]:
# weights and biases
w = np.multiply(alpha.reshape(y.shape[0]), y).dot(x)
b = np.average(y - w.dot(x.T))

# calculation of the accuracy to choose the best params
pred = np.sign(w.dot(x_test.T) + b)
acc = np.sum(pred == y_test) / y_test.shape[0]
pred_tr = np.sign(w.dot(x_tr.T) + b)
acc_tr = np.sum(pred_tr == y_tr) / y_tr.shape[0]

In [ ]:
print("Train accuracy: ", acc_tr)
print("Test accuracy: ", acc)
print("value of m(a) - M(a): ", m - M)

Train accuracy:  0.98125
Test accuracy:  0.9825
value of m(a) - M(a):  0.044616870563102706


# With q = 2 we implemented the most violating pair (MVP) decomposition method

In [ ]:
tr_SVM = customSVM(x_tr, y_tr, opt_C, opt_gamma)
k = 0; num_features = x_tr.shape[1]; q = 2
alpha = np.zeros(y_tr.shape[0])
alpha_app = np.zeros(y_tr.shape[0])
alpha_grad = -1 * np.ones(y_tr.shape[0])
tot_it = 0; tot_fev = 0; tot_time = 0
Q = np.outer(y_tr, y_tr) * tr_SVM.lin_ker()
val_tol = 0.05

In [ ]:
m =1
M = -1

while m - M > val_tol:
    # selcting the two working set
    R_a = np.where(((alpha < opt_C - val_tol) & (y_tr == 1) | (alpha > val_tol) & (y_tr == -1)))[0]
    S_a = np.where(((alpha < opt_C - val_tol) & (y_tr == -1) | (alpha > val_tol) & (y_tr == 1)))[0]
    
    I_a = np.argsort(- alpha_grad[R_a] * y_tr[R_a])[::-1]
    J_a = np.argsort(- alpha_grad[S_a] * y_tr[S_a])
    W = np.concatenate((I_a[: int(q/2)], J_a[: int(q/2)]))
    
    m = - alpha_grad[R_a[I_a[0]]] * y_tr[R_a[I_a[0]]]
    M = - alpha_grad[S_a[J_a[0]]] * y_tr[S_a[J_a[0]]]
    
    # solving the problem for the chosen W
    W_SVM = customSVMlight(x_tr, y_tr, opt_C, opt_gamma, np.concatenate((R_a[I_a[: int(q/2)]], S_a[J_a[: int(q/2)]])), alpha)
    fun_val, n_it, ex_time = W_SVM.opt_alpha()
    tot_it += n_it; tot_time += ex_time
    
    # updating alpha
    x, y, C, gamma, alpha_W = W_SVM.get_par()
    
    for i in range(int(q/2)):
        alpha[R_a[W[i]]] = alpha_W[i]
        alpha[S_a[W[i + int(q/2)]]] = alpha_W[i + int(q/2)]
        
    # computing the new gradient incrementing k and calculating the new m(a) and M(a)
    alpha_grad += (alpha - alpha_app).dot(Q)
        
    for i in range(int(q/2)):
        alpha_app[R_a[W[i]]] = alpha_W[i]
        alpha_app[S_a[W[i + int(q/2)]]] = alpha_W[i + int(q/2)]
    
    k += 1;
print('Iterations:',k)

Iterations: 307


In [ ]:
# weights and biases
w = np.multiply(alpha.reshape(y.shape[0]), y).dot(x)
b = np.average(y - w.dot(x.T))

# calculation of the accuracy to choose the best params
pred = np.sign(w.dot(x_test.T) + b)
acc = np.sum(pred == y_test) / y_test.shape[0]
pred_tr = np.sign(w.dot(x_tr.T) + b)
acc_tr = np.sum(pred_tr == y_tr) / y_tr.shape[0]

In [ ]:
print("Train accuracy: ", acc_tr)
print("Test accuracy: ", acc)
print("value of m(a) - M(a): ", m - M)

Train accuracy:  0.98125
Test accuracy:  0.9825
value of m(a) - M(a):  0.04786468064398686
